# CI/CD 자동화 실습: GitHub Actions & Azure Pipelines

> **실습 환경**: 이 노트북은 GitHub Codespaces에서 실행하도록 구성되어 있습니다.

> **사전 준비**: GitHub 저장소, Azure 구독, ACR, AKS 클러스터가 필요합니다.

이 노트북에서는 GitHub Actions와 Azure Pipelines를 활용하여 컨테이너 기반 애플리케이션의 CI/CD 자동화 실습을 진행합니다.

## 1. CI/CD란?
- **CI(Continuous Integration)**: 코드 변경 사항을 자동으로 빌드 및 테스트
- **CD(Continuous Delivery/Deployment)**: 애플리케이션을 자동으로 배포

### CI/CD의 이점
- 빠른 피드백
- 자동화된 품질 보증
- 신속한 배포

## 2. GitHub Actions로 CI/CD 구현

GitHub Actions는 GitHub 저장소에서 직접 워크플로우를 자동화할 수 있는 도구입니다.

### 2.1 GitHub Actions 워크플로우 구성

GitHub Codespaces에서 `.github/workflows` 디렉터리를 생성하고 워크플로우 파일을 작성합니다.

#### 디렉터리 생성

In [ ]:
# 공통 설정 파일 import
from config import *

# 현재 설정 확인
print_config()

In [ ]:
import subprocess
import os

# GitHub Actions 워크플로우 디렉터리 생성
workflow_dir = ".github/workflows"

if not os.path.exists(workflow_dir):
    os.makedirs(workflow_dir)
    print(f"✅ 디렉터리 생성 완료: {workflow_dir}")
else:
    print(f"✅ 디렉터리가 이미 존재합니다: {workflow_dir}")

# 디렉터리 구조 확인
subprocess.run(["ls", "-la", ".github/"])

#### 워크플로우 파일 작성

`.github/workflows/build-and-push.yml.disabled` 파일을 생성합니다. 사용하려면 `.disabled` 확장자를 제거하세요.

> **주의**: 
> 1. 워크플로우를 활성화하려면 파일 이름에서 `.disabled`를 제거하세요
> 2. GitHub Secrets에 `AZURE_CREDENTIALS`를 설정해야 합니다
> 3. GitHub Variables에 `ACR_NAME`을 설정해야 합니다

In [ ]:
import os

# config.py에서 가져온 변수를 사용하여 워크플로우 파일 생성
if ACR_NAME is None:
    print("⚠️  경고: config.py에 ACR_NAME이 설정되지 않았습니다.")
    print("   01번 노트북에서 ACR을 생성한 후 config.py를 업데이트하세요.")
    acr_name_placeholder = "myacr{timestamp}"
else:
    acr_name_placeholder = ACR_NAME

workflow_content = f"""name: Build and Push to ACR

on:
  push:
    branches: [ main ]
  pull_request:
    branches: [ main ]

jobs:
  build-and-push:
    runs-on: ubuntu-latest
    
    env:
      # ACR 이름 (GitHub Repository Variables에서 설정)
      ACR_NAME: ${{{{ vars.ACR_NAME }}}}
    
    steps:
      - name: Checkout code
        uses: actions/checkout@v3
      
      - name: Set up JDK 21
        uses: actions/setup-java@v3
        with:
          java-version: '21'
          distribution: 'temurin'
      
      - name: Build with Maven
        run: |
          cd springboot-docker-demo
          mvn clean package -Dmaven.test.skip=true
      
      - name: Azure Login
        uses: azure/login@v1
        with:
          creds: ${{{{ secrets.AZURE_CREDENTIALS }}}}
      
      - name: Login to ACR
        run: |
          az acr login --name ${{{{ env.ACR_NAME }}}}
      
      - name: Build and Push Docker Image
        run: |
          cd springboot-docker-demo
          docker build -t ${{{{ env.ACR_NAME }}}}.azurecr.io/myapp:${{{{ github.sha }}}} .
          docker build -t ${{{{ env.ACR_NAME }}}}.azurecr.io/myapp:latest .
          docker push ${{{{ env.ACR_NAME }}}}.azurecr.io/myapp:${{{{ github.sha }}}}
          docker push ${{{{ env.ACR_NAME }}}}.azurecr.io/myapp:latest
"""

# 파일 저장 (.disabled 확장자로)
workflow_file = ".github/workflows/build-and-push.yml.disabled"
os.makedirs(os.path.dirname(workflow_file), exist_ok=True)

with open(workflow_file, "w") as f:
    f.write(workflow_content)

print(f"✅ GitHub Actions 워크플로우 파일 생성 완료: {workflow_file}")
print()
print("⚠️  이 파일은 비활성화 상태입니다.")
print()
print("💡 워크플로우 활성화 방법:")
print("   1. 파일 이름에서 .disabled 제거: build-and-push.yml")
print("   2. GitHub 저장소 > Settings > Secrets and variables > Actions")
print("   3. Secrets 탭에서 AZURE_CREDENTIALS 추가")
print("   4. Variables 탭에서 ACR_NAME 추가 (예: myacr1760169422)")
print()
print("💡 사용된 변수:")
print(f"   APP_NAME: {APP_NAME}")
print(f"   IMAGE_TAG: {IMAGE_TAG}")
print()
print("📄 생성된 파일 내용:")
print()

with open(workflow_file, "r") as f:
    print(f.read())

### 2.2 GitHub Secrets 및 Variables 설정

GitHub 저장소의 Settings > Secrets and variables > Actions에서 다음을 추가하세요:

**Secrets 탭:**
1. **AZURE_CREDENTIALS**: Azure Service Principal 자격 증명

**Variables 탭:**
1. **ACR_NAME**: Azure Container Registry 이름 (예: myacr1760169422)

#### Azure Service Principal 생성

> **참고**: Variables는 민감하지 않은 설정값(ACR 이름 등)에 사용하고, Secrets는 자격 증명에 사용합니다.

In [ ]:
import subprocess
import json

print("🔑 Azure Service Principal 생성")
print("=" * 80)
print()

# 구독 ID 가져오기
print("1️⃣ Azure 구독 정보 확인:")
subscription_result = subprocess.run([
    "az", "account", "show",
    "--query", "{subscriptionId:id, name:name}",
    "-o", "json"
], capture_output=True, text=True)

if subscription_result.returncode == 0:
    try:
        subscription_info = json.loads(subscription_result.stdout)
        subscription_id = subscription_info['subscriptionId']
        subscription_name = subscription_info['name']
        print(f"   ✅ 구독 이름: {subscription_name}")
        print(f"   ✅ 구독 ID: {subscription_id}")
    except:
        subscription_id = "{subscription-id}"
        print("   ⚠️  구독 정보를 가져올 수 없습니다")
else:
    subscription_id = "{subscription-id}"
    print("   ⚠️  구독 정보를 가져올 수 없습니다")

print()

# 리소스 그룹 확인
print("2️⃣ 리소스 그룹 목록:")
rg_result = subprocess.run([
    "az", "group", "list", "--query", "[].name", "-o", "tsv"
], capture_output=True, text=True)

if rg_result.returncode == 0:
    resource_groups = rg_result.stdout.strip().split('\n')
    for rg in resource_groups[:5]:  # 처음 5개만 표시
        print(f"   - {rg}")
    resource_group = resource_groups[0] if resource_groups else "myResourceGroup"
else:
    resource_group = "myResourceGroup"

print()

# Service Principal 생성 명령 생성
sp_name = "github-actions-sp"
sp_command = f"""az ad sp create-for-rbac --name '{sp_name}' --role contributor \\
  --scopes /subscriptions/{subscription_id}/resourceGroups/{resource_group} \\
  --sdk-auth"""

print("3️⃣ Service Principal 생성 명령:")
print()
print(sp_command)
print()

print("=" * 80)
print("💡 다음 단계:")
print("   1. 위 명령을 복사하여 터미널에서 실행")
print("   2. 출력된 JSON 전체를 복사")
print("   3. GitHub 저장소 > Settings > Secrets and variables > Actions")
print()
print("   📝 Secrets 탭에서:")
print("   4. 'New repository secret' 클릭")
print("   5. Name: AZURE_CREDENTIALS")
print("   6. Value: 복사한 JSON 붙여넣기")
print("   7. 'Add secret' 클릭")
print()
print("   📝 Variables 탭에서:")
print("   8. 'New repository variable' 클릭")
print("   9. Name: ACR_NAME")
print(f"   10. Value: {acr_name_placeholder}")
print("   11. 'Add variable' 클릭")
print()
print("   📝 워크플로우 활성화:")
print("   12. build-and-push.yml.disabled 파일의 .disabled 제거")
print()
print("=" * 80)

### 2.3 워크플로우 실행 및 모니터링

코드를 GitHub에 푸시하면 워크플로우가 자동으로 실행됩니다.

1. GitHub Codespaces에서 코드 변경
2. Git commit & push
3. GitHub 저장소의 Actions 탭에서 진행 상황 확인

In [ ]:
import subprocess

print("📤 Git 설정 및 코드 푸시")
print("=" * 80)
print()

# Git 상태 확인
print("1️⃣ Git 상태 확인:")
subprocess.run(["git", "status"])

print()
print("2️⃣ 변경 사항 추가:")
subprocess.run(["git", "add", ".github/workflows/"])

print()
print("3️⃣ 커밋:")
result = subprocess.run([
    "git", "commit", "-m", "Add GitHub Actions workflow (disabled)"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ 커밋 완료")
else:
    print("⚠️  커밋 결과:")
    print(result.stdout)
    print(result.stderr)

print()
print("=" * 80)
print("💡 다음 단계:")
print("   1. git push origin main 실행하여 코드 푸시")
print("   2. GitHub Secrets 및 Variables 설정 (섹션 2.2 참조)")
print("   3. build-and-push.yml.disabled의 .disabled 제거하여 활성화")
print("   4. GitHub 저장소의 Actions 탭에서 워크플로우 실행 확인")
print("   5. URL: https://github.com/{username}/{repo}/actions")
print("=" * 80)

## 3. Azure Pipelines로 CI/CD 구현

Azure Pipelines는 다양한 플랫폼에서 빌드, 테스트, 배포를 자동화할 수 있는 Azure DevOps의 서비스입니다.

### 3.1 Azure DevOps 프로젝트 설정

Azure Pipelines를 사용하기 전에 Azure DevOps에서 프로젝트와 Service Connection을 설정해야 합니다.

#### Azure DevOps 프로젝트 생성
1. [Azure DevOps](https://dev.azure.com/)에 접속
2. 새 프로젝트 생성
3. 프로젝트 이름 입력 (예: aks-mini-labs)

#### 3.2 Azure Pipelines 파일 작성

`azure-pipelines.yml` 파일을 생성하여 빌드 및 배포 파이프라인을 정의합니다.

In [ ]:
import os
import subprocess

# 현재 ACR 이름 확인 (2번 노트북에서 생성한 ACR 사용)
print("📦 ACR 정보 확인 중...")
acr_result = subprocess.run([
    "az", "acr", "list", "--query", "[0].{Name:name, LoginServer:loginServer}", "-o", "json"
], capture_output=True, text=True)

if acr_result.returncode == 0:
    import json
    try:
        acr_info = json.loads(acr_result.stdout)
        ACR_NAME = acr_info['Name']
        ACR_LOGIN_SERVER = acr_info['LoginServer']
        print(f"✅ ACR 이름: {ACR_NAME}")
        print(f"✅ ACR 로그인 서버: {ACR_LOGIN_SERVER}")
    except:
        ACR_NAME = "myacr"
        ACR_LOGIN_SERVER = "myacr.azurecr.io"
        print("⚠️  ACR 정보를 가져올 수 없습니다. 기본값을 사용합니다.")
else:
    ACR_NAME = "myacr"
    ACR_LOGIN_SERVER = "myacr.azurecr.io"
    print("⚠️  ACR 정보를 가져올 수 없습니다. 기본값을 사용합니다.")

print()

# azure-pipelines.yml 파일 생성
pipeline_content = f"""# Azure Pipelines - Build and Push to ACR
trigger:
  branches:
    include:
    - main

pool:
  vmImage: 'ubuntu-latest'

variables:
  # ⚠️ 참고: 이 값들은 노트북 실행 시 자동으로 채워집니다.
  # 수동으로 생성하는 경우 01번 노트북에서 생성한 실제 ACR 이름으로 변경하세요.
  acrName: '{ACR_NAME}'
  acrLoginServer: '{ACR_LOGIN_SERVER}'
  imageRepository: 'myapp'
  dockerfilePath: 'springboot-docker-demo/Dockerfile'

steps:
- checkout: self
  displayName: 'Checkout source code'

- task: Maven@3
  displayName: 'Build with Maven'
  inputs:
    mavenPomFile: 'springboot-docker-demo/pom.xml'
    goals: 'clean package'
    publishJUnitResults: true
    testResultsFiles: '**/surefire-reports/TEST-*.xml'
    javaHomeOption: 'JDKVersion'
    jdkVersionOption: '1.21'

- task: AzureCLI@2
  displayName: 'Build and Push Docker Image to ACR'
  inputs:
    azureSubscription: 'AzureServiceConnection'  # Azure DevOps에서 설정한 Service Connection 이름
    scriptType: 'bash'
    scriptLocation: 'inlineScript'
    inlineScript: |
      # ACR 로그인
      az acr login --name $(acrName)
      
      # 빌드 번호를 태그로 사용
      IMAGE_TAG=$(Build.BuildNumber)
      
      # Docker 이미지 빌드
      cd springboot-docker-demo
      docker build -t $(acrLoginServer)/$(imageRepository):$IMAGE_TAG .
      docker build -t $(acrLoginServer)/$(imageRepository):latest .
      
      # ACR로 푸시
      docker push $(acrLoginServer)/$(imageRepository):$IMAGE_TAG
      docker push $(acrLoginServer)/$(imageRepository):latest
      
      echo "✅ 이미지 푸시 완료: $(acrLoginServer)/$(imageRepository):$IMAGE_TAG"

- task: PublishBuildArtifacts@1
  displayName: 'Publish Kubernetes manifests'
  inputs:
    PathtoPublish: 'deployment.yaml'
    ArtifactName: 'manifests'
"""

# 파일 저장
pipeline_file = "azure-pipelines.yml"

with open(pipeline_file, "w") as f:
    f.write(pipeline_content)

print(f"✅ Azure Pipelines 파일 생성 완료: {pipeline_file}")
print()
print("📄 생성된 파일 내용:")
print()

with open(pipeline_file, "r") as f:
    print(f.read())

### 3.3 Azure Service Connection 설정

Azure Pipelines에서 Azure 리소스에 접근하기 위해 Service Connection을 설정해야 합니다.

In [ ]:
import subprocess

print("🔧 Azure Service Connection 설정 가이드")
print("=" * 80)
print()
print("1️⃣ Azure DevOps에서 Service Connection 생성:")
print("   - Azure DevOps 프로젝트 > Project Settings > Service connections")
print("   - 'New service connection' 클릭")
print("   - 'Azure Resource Manager' 선택")
print("   - 'Service principal (automatic)' 또는 'Service principal (manual)' 선택")
print()
print("2️⃣ Service Connection 이름 설정:")
print("   - Connection name: AzureServiceConnection")
print("   - 구독 및 리소스 그룹 선택")
print("   - 'Save' 클릭")
print()
print("3️⃣ Pipeline 생성:")
print("   - Azure DevOps 프로젝트 > Pipelines > New pipeline")
print("   - GitHub 또는 Azure Repos 선택")
print("   - 저장소 선택")
print("   - 'Existing Azure Pipelines YAML file' 선택")
print("   - Path: /azure-pipelines.yml")
print("   - 'Run' 클릭")
print()
print("=" * 80)
print("💡 참고:")
print("   - Service Connection은 Azure 구독에 대한 인증 정보를 안전하게 저장합니다")
print("   - Pipeline이 자동으로 ACR 접근 권한을 가집니다")
print("=" * 80)

### 3.4 Pipeline 실행 및 모니터링

코드를 저장소에 푸시하면 Azure Pipelines가 자동으로 실행됩니다.

In [ ]:
import subprocess

print("📤 Azure Pipelines 코드 푸시")
print("=" * 80)
print()

# Git 상태 확인
print("1️⃣ Git 상태 확인:")
subprocess.run(["git", "status"])

print()
print("2️⃣ 변경 사항 추가:")
subprocess.run(["git", "add", "azure-pipelines.yml"])

print()
print("3️⃣ 커밋:")
result = subprocess.run([
    "git", "commit", "-m", "Add Azure Pipelines configuration"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✅ 커밋 완료")
else:
    print("⚠️  커밋 결과:")
    print(result.stdout)
    print(result.stderr)

print()
print("=" * 80)
print("💡 다음 단계:")
print("   1. git push origin main 실행하여 코드 푸시")
print("   2. Azure DevOps 프로젝트 > Pipelines에서 자동 실행 확인")
print("   3. 각 단계별 로그 확인 및 모니터링")
print("   4. ACR에 이미지가 푸시되었는지 확인:")
print("      az acr repository list --name {ACR_NAME} --output table")
print("=" * 80)

## 4. ArgoCD를 활용한 GitOps 방식 배포

ArgoCD는 쿠버네티스 환경에서 GitOps 방식을 실현하는 대표적인 오픈소스 도구입니다. Git 저장소의 선언적 매니페스트를 기준으로 애플리케이션을 자동으로 배포 및 동기화합니다.

### GitOps란?
- 모든 배포/운영 구성을 Git 저장소에 선언적으로 관리
- 변경 사항이 감지되면 자동으로 쿠버네티스 클러스터에 적용
- Git을 Single Source of Truth로 사용

### 4.1 ArgoCD 설치

AKS 클러스터에 ArgoCD를 설치합니다.

In [ ]:
import subprocess

print("🚀 ArgoCD 설치 중...")
print("=" * 80)
print()

# 1. argocd 네임스페이스 생성
print("1️⃣ argocd 네임스페이스 생성:")
namespace_result = subprocess.run([
    "kubectl", "create", "namespace", "argocd"
], capture_output=True, text=True)

if "already exists" in namespace_result.stderr:
    print("   ✅ 네임스페이스가 이미 존재합니다")
elif namespace_result.returncode == 0:
    print("   ✅ 네임스페이스 생성 완료")
else:
    print(f"   ⚠️  {namespace_result.stderr}")

print()

# 2. ArgoCD 설치
print("2️⃣ ArgoCD 매니페스트 적용 중...")
print("   (이 작업은 1-2분 정도 소요됩니다)")
install_result = subprocess.run([
    "kubectl", "apply", "-n", "argocd",
    "-f", "https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml"
], capture_output=True, text=True)

if install_result.returncode == 0:
    print("   ✅ ArgoCD 설치 완료")
else:
    print(f"   ❌ 설치 실패: {install_result.stderr}")

print()

# 3. ArgoCD Pod 상태 확인
print("3️⃣ ArgoCD Pod 상태 확인:")
subprocess.run([
    "kubectl", "get", "pods", "-n", "argocd"
])

print()
print("=" * 80)
print("💡 참고:")
print("   - 모든 Pod가 Running 상태가 될 때까지 기다려주세요")
print("   - 상태 확인: kubectl get pods -n argocd -w")
print("=" * 80)

### 4.2 ArgoCD CLI 설치

ArgoCD를 관리하기 위한 CLI 도구를 설치합니다.

In [ ]:
import subprocess
import platform

print("📥 ArgoCD CLI 설치")
print("=" * 80)
print()

# OS 확인
os_type = platform.system()

if os_type == "Darwin":  # macOS
    print("🍎 macOS 감지 - Homebrew로 설치")
    result = subprocess.run([
        "brew", "install", "argocd"
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        print("✅ ArgoCD CLI 설치 완료")
    else:
        print("⚠️  설치 중 오류:")
        print(result.stderr)
        
elif os_type == "Linux":
    print("🐧 Linux 감지 - 바이너리 다운로드")
    print("수동 설치 명령:")
    print("curl -sSL -o /usr/local/bin/argocd https://github.com/argoproj/argo-cd/releases/latest/download/argocd-linux-amd64")
    print("chmod +x /usr/local/bin/argocd")
else:
    print(f"⚠️  지원되지 않는 OS: {os_type}")
    print("   ArgoCD CLI를 수동으로 설치하세요:")
    print("   https://argo-cd.readthedocs.io/en/stable/cli_installation/")

print()

# ArgoCD 버전 확인
print("🔍 ArgoCD CLI 버전 확인:")
version_result = subprocess.run([
    "argocd", "version", "--client"
], capture_output=True, text=True)

if version_result.returncode == 0:
    print(version_result.stdout)
else:
    print("⚠️  ArgoCD CLI가 설치되지 않았습니다")

print()
print("=" * 80)

### 4.3 ArgoCD 서버 접속 설정

ArgoCD UI에 접속하기 위해 초기 패스워드를 확인하고 포트 포워딩을 설정합니다.

In [ ]:
import subprocess
import base64

print("🔐 ArgoCD 초기 패스워드 확인")
print("=" * 80)
print()

# 초기 admin 패스워드 가져오기
print("1️⃣ 초기 admin 패스워드 조회:")
password_result = subprocess.run([
    "kubectl", "get", "secret", "argocd-initial-admin-secret",
    "-n", "argocd",
    "-o", "jsonpath={.data.password}"
], capture_output=True, text=True)

if password_result.returncode == 0 and password_result.stdout:
    # Base64 디코딩
    encoded_password = password_result.stdout
    try:
        decoded_password = base64.b64decode(encoded_password).decode('utf-8')
        print(f"   ✅ Username: admin")
        print(f"   ✅ Password: {decoded_password}")
        print()
        print("   ⚠️  이 패스워드를 안전한 곳에 저장하세요!")
    except:
        print(f"   ⚠️  디코딩 실패. 원본 값: {encoded_password}")
else:
    print("   ❌ 패스워드를 가져올 수 없습니다")
    print("   Secret이 아직 생성되지 않았을 수 있습니다. 잠시 후 다시 시도하세요.")

print()

# 포트 포워딩 안내
print("2️⃣ ArgoCD 서버 접속:")
print("   다음 명령을 별도 터미널에서 실행하세요:")
print()
print("   kubectl port-forward svc/argocd-server -n argocd 8080:443")
print()
print("   그 후 브라우저에서 접속:")
print("   https://localhost:8080")
print()

print("=" * 80)
print("💡 참고:")
print("   - HTTPS 경고가 나타나면 '고급' > '계속 진행' 클릭")
print("   - 로그인 후 Settings > Account에서 패스워드 변경 권장")
print("=" * 80)

### 4.4 ArgoCD 애플리케이션 배포

Git 저장소에 있는 Kubernetes 매니페스트를 ArgoCD로 배포합니다.

> **참고**: 이 예제에서는 현재 저장소의 `deployment.yaml`을 사용합니다.

In [ ]:
import subprocess
import base64

print("🚀 ArgoCD 애플리케이션 생성 및 배포")
print("=" * 80)
print()

# GitHub 저장소 정보 가져오기
print("1️⃣ GitHub 저장소 정보 확인:")
repo_result = subprocess.run([
    "git", "config", "--get", "remote.origin.url"
], capture_output=True, text=True)

if repo_result.returncode == 0:
    repo_url = repo_result.stdout.strip()
    print(f"   ✅ 저장소 URL: {repo_url}")
else:
    repo_url = "https://github.com/your-org/your-repo.git"
    print(f"   ⚠️  저장소 URL을 가져올 수 없습니다. 기본값 사용: {repo_url}")

print()

# ArgoCD 패스워드 확인
print("2️⃣ ArgoCD 초기 패스워드 확인:")
password_result = subprocess.run([
    "kubectl", "get", "secret", "argocd-initial-admin-secret",
    "-n", "argocd",
    "-o", "jsonpath={.data.password}"
], capture_output=True, text=True)

if password_result.returncode == 0 and password_result.stdout:
    encoded_password = password_result.stdout
    try:
        admin_password = base64.b64decode(encoded_password).decode('utf-8')
        print(f"   ✅ Admin Password: {admin_password}")
    except:
        admin_password = None
        print(f"   ⚠️  패스워드 디코딩 실패")
else:
    admin_password = None
    print("   ⚠️  패스워드를 가져올 수 없습니다")

print()

# ArgoCD 로그인 가이드
print("3️⃣ ArgoCD CLI 로그인:")
print("   먼저 별도 터미널에서 포트 포워딩을 실행하세요:")
print()
print("   kubectl port-forward svc/argocd-server -n argocd 8080:443")
print()
print("   그 후 ArgoCD에 로그인:")
print()
if admin_password:
    print(f"   argocd login localhost:8080 --username admin --password {admin_password} --insecure")
else:
    print(f"   argocd login localhost:8080 --username admin --insecure")
    print("   (패스워드 입력 요청 시 위에서 확인한 패스워드 입력)")
print()

# ArgoCD 애플리케이션 생성 가이드
print("4️⃣ ArgoCD CLI로 애플리케이션 생성:")
print("   로그인 후 다음 명령을 실행하세요:")
print()
print(f"   argocd app create myapp \\")
print(f"     --repo {repo_url} \\")
print(f"     --path . \\")
print(f"     --dest-server https://kubernetes.default.svc \\")
print(f"     --dest-namespace default")
print()

print("5️⃣ 애플리케이션 동기화:")
print("   argocd app sync myapp")
print()

print("6️⃣ 애플리케이션 상태 확인:")
print("   argocd app get myapp")
print()

print("=" * 80)
print("💡 ArgoCD UI에서 확인:")
print("   1. https://localhost:8080 접속")
print("   2. Applications 메뉴에서 myapp 확인")
print("   3. 애플리케이션 클릭하여 상세 정보 및 리소스 상태 확인")
print("   4. Git 저장소 변경 시 자동 동기화 확인")
print()
print("⚠️  중요: 먼저 'argocd login' 명령을 실행해야 합니다!")
print("=" * 80)

### 4.5 GitOps 워크플로우 테스트

Git 저장소의 매니페스트를 변경하면 ArgoCD가 자동으로 감지하고 동기화합니다.

In [ ]:
import subprocess

print("🧪 GitOps 워크플로우 테스트")
print("=" * 80)
print()

print("📝 시나리오: deployment.yaml의 replica 수를 변경하여 자동 동기화 테스트")
print()

# 현재 deployment.yaml 확인
print("1️⃣ 현재 deployment.yaml 확인:")
subprocess.run(["grep", "-A", "2", "replicas:", "deployment.yaml"])

print()
print("2️⃣ replica 수 변경 (예: 2 → 3):")
print("   deployment.yaml 파일을 편집하세요:")
print("   - spec.replicas 값을 변경")
print()

print("3️⃣ 변경사항 커밋 및 푸시:")
print("   git add deployment.yaml")
print("   git commit -m 'Update replicas to 3'")
print("   git push origin main")
print()

print("4️⃣ ArgoCD 동기화 확인:")
print("   - ArgoCD UI에서 자동 감지 확인 (약 3분 소요)")
print("   - 또는 수동 동기화: argocd app sync myapp")
print()

print("5️⃣ Pod 수 변경 확인:")
print("   kubectl get pods")
print()

print("=" * 80)
print("💡 GitOps의 장점:")
print("   ✅ Git이 배포 상태의 Single Source of Truth")
print("   ✅ 모든 변경 이력이 Git에 기록됨")
print("   ✅ 롤백이 간단함 (git revert)")
print("   ✅ 선언적 구성으로 일관성 보장")
print("=" * 80)

## 5. 정리

이번 실습에서는 GitHub Actions, Azure Pipelines, ArgoCD를 활용해 컨테이너 이미지 빌드부터 ACR 푸시, AKS 배포, GitOps 기반 선언적 배포까지 CI/CD 자동화 전 과정을 GitHub Codespaces 환경에서 실습했습니다.

### 실습한 내용 요약
- ✅ CI/CD 개념 이해
- ✅ GitHub Actions 워크플로우 구성
- ✅ Azure Pipelines 파이프라인 구성
- ✅ ArgoCD를 활용한 GitOps 배포
- ✅ 자동화된 빌드, 테스트, 배포 파이프라인 구축

### 주요 학습 포인트
- **GitHub Actions**: 코드 변경 시 자동으로 빌드 및 ACR 푸시
- **Azure Pipelines**: Azure DevOps를 통한 엔터프라이즈급 CI/CD
- **ArgoCD**: Git 저장소 기반 선언적 배포 및 동기화

이제 컨테이너 이미지를 자동으로 빌드하고, ACR로 푸시하며, AKS에 배포하는 전체 자동화 파이프라인과 GitOps 기반 배포까지 구축할 수 있습니다.

---

## 🎉 실습 완료!

축하합니다! AKS Mini Labs의 모든 실습을 완료했습니다.

- 컨테이너 기본 개념부터 실제 운영 환경 구축까지
- Azure의 관리형 서비스를 활용한 효율적인 클라우드 네이티브 애플리케이션 개발
- CI/CD 자동화를 통한 DevOps 실무 역량 향상

> **GitHub Codespaces** 덕분에 로컬 환경 설정 없이 모든 실습을 진행할 수 있었습니다!